In [10]:
import pandas
dataSource = pandas.read_csv('weight_height_10000.csv')

In [11]:
data = dataSource
data['Height'] = dataSource['Height'].multiply(2.54)
data['Weight'] = dataSource['Weight'].multiply(0.45359237)
print(data.head())
print(len(data))

  Gender      Height      Weight
0   Male  187.571423  109.721075
1   Male  174.706036   73.622792
2   Male  188.239668   96.497629
3   Male  182.196685   99.809586
4   Male  177.499761   93.598695
9996


In [12]:
import numpy as np

# Chuẩn hóa dữ liệu
heights_norm = (data['Height'].to_numpy() - np.mean(data['Height'].to_numpy())) / np.std(data['Height'].to_numpy())
weights_norm = (data['Weight'].to_numpy() - np.mean(data['Weight'].to_numpy())) / np.std(data['Weight'].to_numpy())

# Xây dựng mô hình MLP
input_size = 1
hidden_size = 10
output_size = 1
learning_rate = 0.01

# Khởi tạo trọng số
np.random.seed(0)
W1 = np.random.randn(hidden_size, input_size) * 0.01
b1 = np.zeros((hidden_size, 1))
W2 = np.random.randn(output_size, hidden_size) * 0.01
b2 = np.zeros((output_size, 1))


# Huấn luyện mô hình
num_epochs = 100
for epoch in range(num_epochs):
    # Forward pass
    Z1 = np.dot(W1, heights_norm.reshape(1, -1)) + b1
    A1 = np.maximum(0, Z1)  # ReLU activation
    Z2 = np.dot(W2, A1) + b2
    predictions = Z2

    # Compute loss
    loss = np.mean((predictions - weights_norm.reshape(1, -1))**2)

    # Backward pass
    dZ2 = predictions - weights_norm.reshape(1, -1)
    dW2 = np.dot(dZ2, A1.T) / data['Height'].shape[0]
    db2 = np.mean(dZ2, axis=1, keepdims=True)
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = dA1 * (Z1 > 0)  # ReLU gradient
    dW1 = np.dot(dZ1, heights_norm.reshape(1, -1).T) / data['Height'].shape[0]
    db1 = np.mean(dZ1, axis=1, keepdims=True)

    # Update weights
    W1 -= learning_rate * dW1
    b1 -= learning_rate * db1
    W2 -= learning_rate * dW2
    b2 -= learning_rate * db2

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss:.4f}')

# Dự đoán cân nặng cho một chiều cao mới
new_height = 165
new_height_norm = (new_height - np.mean(data['Height'])) / np.std(data['Height'])
Z1 = np.dot(W1, new_height_norm) + b1
A1 = np.maximum(0, Z1)  # ReLU activation
Z2 = np.dot(W2, A1) + b2
predicted_weight_norm = Z2
predicted_weight = predicted_weight_norm * np.std(data['Weight']) + np.mean(data['Weight'])
print(f'Cân nặng dự đoán cho chiều cao {new_height} cm là: {predicted_weight.item()} kg')

Epoch [100/100], Loss: 0.9981
Cân nặng dự đoán cho chiều cao 165 cm là: 73.2225817468812 kg
